In [1]:
import pandas as pd
import numpy as np

# --- PARAMETRI DI CONFIGURAZIONE ---
NUM_PARTICELLE = 10000  # Numero totale di particelle da simulare
RAGGIO_DIMORPHOS = 80.0 # Raggio approssimativo di Dimorphos in metri
FILE_OUTPUT = 'dummy_ejecta_dataset.csv'

print(f"Inizio la generazione di un dataset con {NUM_PARTICELLE} particelle...")

# 1. GENERAZIONE DEGLI ID
# Un semplice array di interi univoci per ogni particella.
ids = np.arange(1, NUM_PARTICELLE + 1)

# 2. GENERAZIONE DELLE POSIZIONI INIZIALI (x, y, z)
# Simuliamo che le particelle vengano eiettate dalla superficie di una sfera.
# Generiamo punti casuali in 3D e poi li normalizziamo per proiettarli sulla
# superficie della sfera con raggio RAGGIO_DIMORPHOS.
posizioni_raw = np.random.randn(NUM_PARTICELLE, 3)  # Distribuzione normale standard
norme = np.linalg.norm(posizioni_raw, axis=1, keepdims=True)
posizioni_normalizzate = posizioni_raw / norme
posizioni = posizioni_normalizzate * RAGGIO_DIMORPHOS
# Aggiungiamo un piccolo rumore per non essere perfettamente sulla sfera
posizioni += np.random.normal(scale=0.1, size=posizioni.shape)

# 3. GENERAZIONE DELLE VELOCITÀ INIZIALI (vx, vy, vz)
# Assumiamo che la velocità sia principalmente radiale (verso l'esterno).
# La direzione di base è il vettore posizione normalizzato.
# La magnitudine della velocità (speed) segue una distribuzione non uniforme,
# con più particelle lente che veloci.
magnitudini_velocita = np.random.lognormal(mean=2.5, sigma=1.0, size=NUM_PARTICELLE)
# Creiamo i vettori velocità scalando la direzione per la magnitudine
velocita = posizioni_normalizzate * magnitudini_velocita[:, np.newaxis]
# Aggiungiamo una componente di "rumore" per variare la direzione e non renderla
# perfettamente radiale, simulando un cono di eiezione.
velocita += np.random.normal(scale=0.5, size=velocita.shape)

# 4. GENERAZIONE DELLA DIMENSIONE DELLE PARTICELLE
# Le dimensioni degli ejecta da impatto spesso seguono una legge di potenza
# (molte particelle piccole, poche grandi). La distribuzione di Pareto è ideale.
# Il parametro 'a' controlla la pendenza della distribuzione.
# Aggiungiamo 0.001 m (1 mm) come dimensione minima.
dimensioni = (np.random.pareto(a=1.5, size=NUM_PARTICELLE)) + 0.001

# 5. GENERAZIONE DEL DESTINO FINALE (Etichetta di Classificazione)
# Questa è la colonna target. La creiamo basandoci su regole plausibili,
# principalmente legate alla velocità iniziale.
# - Velocità molto alte -> Fuga (escape)
# - Velocità molto basse -> Impatto (ricaduta sul corpo)
# - Velocità intermedie -> Sopravvivenza (in orbita)
fates = []
V_IMPATTO_MAX = 5.0    # Velocità sotto la quale è probabile l'impatto
V_FUGA_MIN = 30.0   # Velocità sopra la quale è probabile la fuga

for v in magnitudini_velocita:
    if v < V_IMPATTO_MAX:
        # Aggiungiamo casualità: 90% di probabilità di impatto, 10% sopravvivenza
        destino = np.random.choice(['impatto', 'sopravvivenza'], p=[0.9, 0.1])
    elif v > V_FUGA_MIN:
        # 95% di probabilità di fuga, 5% sopravvivenza
        destino = np.random.choice(['fuga', 'sopravvivenza'], p=[0.95, 0.05])
    else:
        # Zona intermedia: più probabile la sopravvivenza
        destino = np.random.choice(['sopravvivenza', 'impatto', 'fuga'], p=[0.8, 0.1, 0.1])
    fates.append(destino)

# 6. CREAZIONE DEL DATAFRAME PANDAS
df = pd.DataFrame({
    'id_particella': ids,
    'pos_x_m': posizioni[:, 0],
    'pos_y_m': posizioni[:, 1],
    'pos_z_m': posizioni[:, 2],
    'vel_x_ms': velocita[:, 0],
    'vel_y_ms': velocita[:, 1],
    'vel_z_ms': velocita[:, 2],
    'dimensione_m': dimensioni,
    'destino': fates
})

# Arrotondiamo i valori per una migliore leggibilità
df = df.round(4)

# 7. SALVATAGGIO IN FORMATO CSV
df.to_csv(FILE_OUTPUT, index=False)

print(f"\nDataset generato e salvato con successo in '{FILE_OUTPUT}'")
print("\n--- Anteprima del Dataset (prime 5 righe): ---")
print(df.head())
print("\n--- Distribuzione delle etichette 'destino': ---")
print(df['destino'].value_counts())

Inizio la generazione di un dataset con 10000 particelle...

Dataset generato e salvato con successo in 'dummy_ejecta_dataset.csv'

--- Anteprima del Dataset (prime 5 righe): ---
   id_particella  pos_x_m  pos_y_m  pos_z_m  vel_x_ms  vel_y_ms  vel_z_ms  \
0              1 -11.3109  78.8904  -7.9005   -1.8683    9.7984   -1.2443   
1              2   1.2717 -79.8519   2.6902   -0.6585   -5.7420   -0.0210   
2              3 -11.3773 -57.1442 -54.9293   -8.0124  -36.7707  -35.8474   
3              4 -11.7605  61.2306 -50.1882   -2.1673   10.5401   -8.7905   
4              5   6.3496  51.8048 -60.6539   -0.0990   12.5296  -15.1152   

   dimensione_m        destino  
0        0.0210  sopravvivenza  
1        0.0763  sopravvivenza  
2        1.1618           fuga  
3        6.3322  sopravvivenza  
4        0.0341           fuga  

--- Distribuzione delle etichette 'destino': ---
destino
sopravvivenza    5351
fuga             2401
impatto          2248
Name: count, dtype: int64
